In [ ]:
# setup scratch area to explore data
! mkdir -p ../scratch

# download kaggle archive
# https://www.kaggle.com/datasets/misrakahmed/vegetable-image-dataset?resource=download

# extract kaggle dataset
! cd ../scratch; unzip -n archive.zip

## Archived

Archived Code Beyond This Point

---

In [ ]:
import os
import boto3

local_dest_dir = os.path.join(os.getcwd(), "TrainingData/")


rmtree(local_dest_dir)
os.makedirs(local_dest_dir)

import urllib3

urllib3.disable_warnings()

# Fetch all env variable these values will be fetched from Data Connection setup

access_key = os.getenv("AWS_ACCESS_KEY_ID")
secret = os.getenv("AWS_SECRET_ACCESS_KEY")
s3endpoint = os.getenv("AWS_S3_ENDPOINT")
bucket_name = os.getenv("AWS_S3_BUCKET")


"""
 Download all files from local MinIO server to Directory TrainingData
"""

s3 = boto3.resource(
    "s3",
    aws_access_key_id=access_key,
    aws_secret_access_key=secret,
    config=boto3.session.Config(signature_version="s3v4"),
    verify=False,
    endpoint_url=s3endpoint,
)
s3_client = s3.meta.client

bucket = s3.Bucket(bucket_name)

objects = bucket.objects.filter(Prefix="/")
for obj in objects:
    path, filename = os.path.split(obj.key)
    if len(path) != 0 and not os.path.exists(os.path.join(local_dest_dir, path)):
        os.makedirs(os.path.join(local_dest_dir, path))
    if not obj.key.endswith("/"):
        download_to = local_dest_dir + path + "/" + filename if path else filename
        s3_client.download_file(bucket_name, obj.key, download_to)

print("Download Completed")